In [15]:
from simpletransformers.classification import ClassificationModel
import numpy as np
import pandas as pd
from sklearn import preprocessing
from scipy import stats
import wandb


In [16]:
#subscribers data
daryo_n = [597660, 635395, 876932, 944683, 970976, 974959, 1001620, 987818, 955630, 930500]
kun_n = [866120, 897421, 1234502, 1327874, 1400304, 1419048, 1468400, 1467269, 1487835, 1533702]
qalampir_n = [84189, 91990, 132252, 141007, 162540, 167763, 182913, 186740, 185759, 183327]

view_period=7 # days 

In [17]:
def create_fields(data, source):
    dates = pd.DatetimeIndex(data['date'])
    n_views = data['num_views'].values

    labels = []
    day_of_week = []
    for i in range(0, len(dates)):

        daily_rate = n_views[i]/view_period
        average_daily_rate = int(source[dates.month[i]-1]/100) # scaling down it a bit

        labels.append(daily_rate/average_daily_rate)
        day_of_week.append(dates[i].dayofweek)
    
    return labels, day_of_week

In [18]:
daryo = pd.read_json("daryo_articles.jsonl", orient='columns', lines=False)
kun = pd.read_json("kun_articles.jsonl", orient='columns', lines=True)
qalampir = pd.read_json("qalampir_articles.jsonl", orient='columns', lines=True)

labels, day_of_week = create_fields(daryo, daryo_n)
    
daryo.insert(len(daryo.columns), "labels", labels, True) 
daryo.insert(len(daryo.columns)-1, "day_of_week", day_of_week, True)
daryo.insert(2, "source", len(daryo)*[1], True) 

labels, day_of_week = create_fields(kun, kun_n)
    
kun.insert(len(kun.columns), "labels", labels, True) 
kun.insert(len(kun.columns)-1, "day_of_week", day_of_week, True)
kun.insert(2, "source", len(kun)*[2], True) 

labels, day_of_week = create_fields(qalampir, qalampir_n)
    
qalampir.insert(len(qalampir.columns), "labels", labels, True) 
qalampir.insert(len(qalampir.columns)-1, "day_of_week", day_of_week, True)
qalampir.insert(2, "source", len(qalampir)*[3], True) 


In [22]:
# output = [daryo, kun, qalampir]
# output = pd.concat(output)
# output.to_json('output_raw.jsonl', orient='records', lines=True)
set(qalampir.category.values)



{'3-саҳифа',
 'Бу қизиқ',
 'Жамият',
 'Интервью',
 'Маданият-санъат',
 'Мусиқа',
 'Олам',
 'Реклама',
 'Саломатлик',
 'Спорт',
 'Таҳлил',
 'Технология',
 'Фойдали'}

In [9]:
def _format(data):
    return pd.DataFrame({
        'text_a': '[CLS] ' + data['content'],
        'text_b': data['title'],
        'text_c': data['num_links'],
        'text_d': data['num_images'],
        'text_e': data['day_of_week'],
        'text_f': data['num_quotes'],
        'labels': data['labels']
    })

In [10]:
def scale(train_df, column):
    return (train_df[column]-train_df[column].min())/(train_df[column].max()-train_df[column].min())


In [11]:
# scale it
def scale_fields(train_df):
    train_df['text_c'] = scale(train_df, 'text_c')
    train_df['text_d'] = scale(train_df, 'text_d')
    train_df['text_e'] = scale(train_df, 'text_e')
    train_df['text_f'] = scale(train_df, 'text_f')
    train_df['labels'] = scale(train_df, 'labels')
    return train_df



In [12]:
daryo = scale_fields(_format(daryo))
kun = scale_fields(_format(kun))
qalampir = scale_fields(_format(qalampir))

train_df = [daryo, kun, qalampir]
train_df = pd.concat(train_df)

In [14]:
# train_df.sort_values(by=['labels'], ascending=False)
# train_df['labels'].mean()
#train_df.to_json('output_raw.jsonl', orient='records', lines=True)
train_df
features = ['content', 'title', 'num_links', 'num_images', 'day_of_week', 'num_quotes', 'label']

,text_a,text_b,text_c,text_d,text_e,text_f,labels
0,[CLS] Microsoft kompaniyasi asoschisi Bill Gey...,Bill Geyts koronavirus bilan bog‘liq vaziyat y...,0.042857,0.008065,0.666667,0.111111,0.025070
1,[CLS] Sog‘liqni saqlash vazirining tegishli bu...,Respublika akusherlik va ginekologiya markazig...,0.057143,0.008065,0.666667,0.000000,0.031297
2,[CLS] Forbes Uzbekistan o‘z faoliyatini to‘xta...,Forbes Uzbekistan o‘z faoliyatini to‘xtatdi,0.057143,0.008065,0.666667,0.000000,0.039010
3,[CLS] Transport vazirligi epidemiologik vaziya...,Toshkentda yo‘lovchilarga jamoat transportidan...,0.028571,0.008065,0.666667,0.000000,0.026868
4,[CLS] Sog‘liqni saqlash vazirligi COVID-19 kor...,SSV: Koronavirusga chalingan bemorlarni davo s...,0.057143,0.008065,0.666667,0.000000,0.027063
...,...,...,...,...,...,...,...
9920,[CLS] Германия канцлери Ангела Меркелнинг таък...,Меркель Европа мамлакатларини “уйғонишга” чақирди,0.052632,0.020000,0.833333,0.100000,0.030054
9921,[CLS] Шу кунларда ИИВнинг “босар-тусарини билм...,Депутат: “Айрим ОАВлар наманганлик депутат воқ...,0.052632,0.020000,0.000000,0.200000,0.069785
9922,[CLS] Дунё бўйлаб коронавирусга чалинганлар 80...,Коронавирусга чалинганлар 800 000 дан ошди,0.315789,0.020000,0.166667,0.000000,0.043981
9923,"[CLS] Аввалроқ, Хоразм вилоятида янги ташкил э...",Тупроққалъа тумани “тўртлиги” тайинланди,0.052632,0.040000,0.000000,0.000000,0.023006


In [24]:
from sklearn.model_selection import train_test_split
train_df = train_df.sample(frac=1)
train, test = train_test_split(train_df, test_size=0.2)

In [25]:
hyperparameter_defaults = dict(
    bs = 32,
    lr = 4e-5,
    epochs = 1,
    sequence_length = 512,
)

wandb.init(config= hyperparameter_defaults,project="duvduvgap")
config = wandb.config
wandb.config.experiment = "roberta"

train_args={
    "output_dir": "outputs/roberta",
    "cache_dir": "cache/",
    "best_model_dir": "outputs/best_model/roberta",
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'num_train_epochs': config.epochs,
    'regression': True,
    'max_seq_length': config.sequence_length,
    'n_gpu': 3,
    "learning_rate": config.lr,
    "train_batch_size": config.bs,
    "eval_batch_size": config.bs,
}

# Create a ClassificationModel
model = ClassificationModel("roberta", "roberta-large", num_labels=1, use_cuda=True, args=train_args)

# Train the model
model.train_model(train)

wandb: Wandb version 0.10.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly init

In [26]:
result, model_outputs, wrong_predictions = model.eval_model(test)
test.insert(len(test.columns), "prediction", model_outputs, True) 
test = test.sort_values(by=['prediction'], ascending=False)

In [28]:
print(result)

{'eval_loss': 0.004227833283103762}


In [29]:
def analysis(data):
    print(f"Correlation between labels and predictions: {stats.pearsonr(data['labels'], data['prediction'])}\n")
    print(f"Number of links and predictions: {stats.pearsonr(data['text_c'], data['prediction'])}\n")
    print(f"Number of images and predictions: {stats.pearsonr(data['text_d'], data['prediction'])}\n")
    print(f"Day of the week and predictions: {stats.pearsonr(data['text_e'], data['prediction'])}\n")
    print(f"Number of quotes and predictions: {stats.pearsonr(data['text_f'], data['prediction'])}\n")


In [30]:
analysis(test)

Correlation between labels and predictions: (-0.014369583856630508, 0.31251007056521707)

Number of links and predictions: (0.026002031036423685, 0.06758366497971811)

Number of images and predictions: (0.056583262387460705, 6.892989585565515e-05)

Day of the week and predictions: (-0.010658954945727031, 0.45376713957654535)

Number of quotes and predictions: (0.05961313989492446, 2.7476186087366805e-05)

